In [9]:
# 라이브러리 import
import gensim
import multiprocessing
import threading
import pandas as pd

# CPU 수를 print
print(multiprocessing.cpu_count())

8


In [10]:
# 데이터 로드
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# 데이터 행개수 확인 (총 100,000 개)
print (("Read {0} labeled train reviews, {1} labeled test reviews, "+ '\n' +"and {2} unlabeled reviews").format(train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, 
and 50000 unlabeled reviews


In [8]:
# head로 확인
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [12]:
# head로 확인
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [23]:
unlabeled_train.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [11]:
# 리뷰 형식 확인
train["review"][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [17]:
# 문자열 전처리를 위한 라이브러리 import
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # document를 list화
    # 불용어를 없애는 것은 선택, 보통 높은 질의 워드벡터를 얻기 위해 불용어를 제거하지는 않음.
    
    # HTML 함수 지우기
    review_text = BeautifulSoup(review).get_text()
      
    # 글자가 아닌것들 없애기 []안의 ^는 not과 같음.
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 소문자화 후 띄어쓰기를 기준으로 모두 나눔.
    words = review_text.lower().split()
    
    # 불용어 제거
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    # 단어 리스트 반환
    return(words)

In [13]:
# document를 문장 단위로 나누기위해 nltk 데이터 중 punkt 다운로드
import nltk.data
nltk.download()   # 데이터 다운로드

# punkt tokenizer (문장별로 나누는 토큰)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# document를 문장으로 나누는 함수
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # 일단 문장별로 리스트가 만들어지고, 각 문장리스트 안에 단어로 구성된 리스트로 반환
    
    # punkt tokenizer를 통해 document를 문장으로 나눔
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 각 문장에 대해 for문을 돌림
    sentences = []
    for raw_sentence in raw_sentences:
        # 만약 문장리스트안에 단어가 없다면 스킵
        if len(raw_sentence) > 0:
            # 문장리스트 안에 단어가 있다면 사전정의한 review_to_wordlist를 통해 단어별로 분절
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    
    # 문장리스트반환 (리스트 안 리스트 형식)
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [21]:
[['a'],['a']]+ [['b'],['b']]

[['a'], ['a'], ['b'], ['b']]

In [18]:
sentences = []  # 문장을 넣을 빈 리스트 생성

print ("Parsing sentences from training set")

# 하나의 리뷰(document)마다 문장리스트를 review_to setence로 만듬
# for문으로 각 리뷰를 한번 돌때마다 생성된 문장리스트가 sentences 리스트 뒤에 붙음.
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


D:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file D:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
D:\python\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
D:\python\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful

In [22]:
# 4번째 문장리스트 확인
sentences[4]

['some',
 'may',
 'call',
 'mj',
 'an',
 'egotist',
 'for',
 'consenting',
 'to',
 'the',
 'making',
 'of',
 'this',
 'movie',
 'but',
 'mj',
 'and',
 'most',
 'of',
 'his',
 'fans',
 'would',
 'say',
 'that',
 'he',
 'made',
 'it',
 'for',
 'the',
 'fans',
 'which',
 'if',
 'true',
 'is',
 'really',
 'nice',
 'of',
 'him',
 'the',
 'actual',
 'feature',
 'film',
 'bit',
 'when',
 'it',
 'finally',
 'starts',
 'is',
 'only',
 'on',
 'for',
 'minutes',
 'or',
 'so',
 'excluding',
 'the',
 'smooth',
 'criminal',
 'sequence',
 'and',
 'joe',
 'pesci',
 'is',
 'convincing',
 'as',
 'a',
 'psychopathic',
 'all',
 'powerful',
 'drug',
 'lord']

In [15]:
print ("Parsing sentences from unlabeled set")

# 라벨없는 훈련셋을 문장리스트로 추가 변환하여 sentence 리스트에 붙임

for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from unlabeled set


D:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file D:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
D:\python\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
D:\python\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an

In [16]:
# 총 문장리스트가 약 850,000개 정도
print (len(sentences))
print (sentences[0])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [17]:
print (sentences[1])

['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [19]:
# https://radimrehurek.com/gensim/models/word2vec.html
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Word2Vec 파라미터 설정
num_features = 300    # 워드벡터 차원수 설정
min_word_count = 40   # 최소 단어수
num_workers = 6       # 병렬처리시 쓰레드 수
context = 10          # CBOW 모델시 input인 주변단어개수(window) 설정
downsampling = 1e-3   # 이 비율보다 더 높은 빈도의 단어들 중 랜덤하게 downsample

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")

# CBOW 모델
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# 더 이상의 훈련을 하지 않을 것이라면,
# L2 정규화벡터로 변환을 시켜 메모리를 절약 수 있다. 변환후에는 훈련을 할수 없으니 주의.
model.init_sims(replace=True)

# 모델명을 정한 후, 워드벡터를 저장하여 나중에 Word2Vec.load()를 사용해, 다시 훈련하지 않아도 로드하여 바로 사용할 수 있다.
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-06-23 23:33:39,976 : INFO : collecting all words and their counts
2017-06-23 23:33:39,977 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-23 23:33:40,023 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2017-06-23 23:33:40,066 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2017-06-23 23:33:40,108 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2017-06-23 23:33:40,152 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types


Training model...


2017-06-23 23:33:40,199 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2017-06-23 23:33:40,242 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2017-06-23 23:33:40,287 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2017-06-23 23:33:40,333 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2017-06-23 23:33:40,377 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2017-06-23 23:33:40,421 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2017-06-23 23:33:40,464 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2017-06-23 23:33:40,508 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keeping 54119 word types
2017-06-23 23:33:40,554 : INFO : PROGRESS: at sentence #130000, processed 2894303 words, keep

In [20]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [21]:
model.most_similar("man")

[('woman', 0.6157041788101196),
 ('lady', 0.5912197828292847),
 ('monk', 0.5590580701828003),
 ('lad', 0.5582822561264038),
 ('men', 0.5162534713745117),
 ('guy', 0.5106900930404663),
 ('millionaire', 0.5048726797103882),
 ('farmer', 0.5016376972198486),
 ('businessman', 0.500268816947937),
 ('person', 0.4969886541366577)]

In [28]:
model.syn1neg[1:10]

array([[ 0.63299465, -0.14457166,  0.32823831, ...,  0.36432058,
         0.2555986 ,  0.03019164],
       [ 0.81615639, -0.15518148,  0.43223628, ...,  0.33658126,
         0.3930881 ,  0.05845106],
       [ 0.91632974, -0.1225011 ,  0.47338527, ...,  0.44294769,
         0.42992458,  0.12480053],
       ..., 
       [ 0.87911153, -0.19573618,  0.23994713, ...,  0.18786989,
         0.48512921, -0.25085855],
       [ 0.98206949, -0.22142909,  0.53364044, ...,  0.33518466,
         0.4603622 , -0.22779191],
       [ 0.67038852, -0.13886328,  0.45360562, ...,  0.35519555,
         0.26664361,  0.0631773 ]], dtype=float32)